In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [2]:
path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 600893
total chars: 57


In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

total chars: 57
nb sequences: 200285
Vectorization...


In [4]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [5]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [6]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
200285/200285 [==============================] - 57s 283us/step - loss: 1.9737

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " is young, prizes metaphysical
explanati"
 is young, prizes metaphysical
explanations and an an a lite the our the inderstous of the sension of the and a moral and an a respection of all the higher and the concernting and and a sensions of the our the conscience of the order of the our in the cannot and the sensing of the conscience of the senses of the interplete the cannot and soul of the conscience of the our the our the resenting of the and our the higher our the our the wa
----- diversity: 0.5
----- Generating with seed: " is young, prizes metaphysical
explanati"
 is young, prizes metaphysical
explanation of the cannot our other which as the in his been be its upon all theme the order its lateress, with the only and lithly and conscience of respinces
the have
an and conseive there to the
pain inte, and who

C:\Users\alibe\anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


e contectionfulness, as a fleed to resides, whothall priinilyor-
Epoch 18/60
200285/200285 [==============================] - 55s 272us/step - loss: 1.3183

----- Generating text after Epoch: 17
----- diversity: 0.2
----- Generating with seed: "), just as he stands in the presence of "
), just as he stands in the presence of the spirit and an extent to the other of the self see and the possibility of the spirit of the possibility of the spirit the spirit the their of the spirit in the extent of the possibility of the spirit of the spirit and soul of the procrarted and the spirit of the spirit of the spirit and the spirit of the philosophers of the spirit and all the procrarter of the spirit the self-read to the spirit
----- diversity: 0.5
----- Generating with seed: "), just as he stands in the presence of "
), just as he stands in the presence of the freedom of a stronged with the thing and intellect of the standing of the higher conceptions of the radiations of the end, we live is th